In [7]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append("..")  # if simulator.ipynb is in /notebooks/

from simulation.core import System
from simulation.controller import TrafficController
from simulation.data_logger import DataLogger
from simulation.core import Road, Obstacle
from simulation.spawners import timed_spawner, density_random_lane_spawner
from helper import plot_average_velocity_per_lane, plot_lane_occupancy, compute_avg_velocity_over_time, plot_lane_traffic_stats


In [6]:
def get_last_frame_average_velocity(df):
    if df.empty:
        print("No data logged.")
        return None
    last_time = df["time"].max()
    last_frame = df[df["time"] == last_time]
    return last_frame["velocity"].mean()

In [ ]:
dt = 0.1
sim_time = 400

num_cars = 100
starting_speed = 60

road_length = 2000
max_speed = 60

num_lanes = 5

def test_loop(n, rate, starting_value, should_print = False):

    dataframes = []

    for _ in range(n):

        system = System(dt=dt, final_time=sim_time)

        start_pos = (0, 300)
        end_pos = (road_length, 300)
        road = Road(num_lanes, max_speed, start_pos, end_pos)

        system.add_road(road)
        logger = DataLogger(tag="basic_one_road_test")
        controller = TrafficController(system, logger=logger)

        controller.add_spawn_rule(density_random_lane_spawner(starting_value, 0, num_cars, speed=starting_speed))

        system.block_lane_from_offsets(0, 0, 500, 700)
        system.block_lane_from_offsets(0, 4, 350, 450)


        for _ in range(int(sim_time / dt)):
            end = controller.control(dt, should_print=should_print)
            system.update(logger)
    
            if end:
                break

        df = logger.to_dataframe()
        dataframes.append(df)
        print("Logging to dataframe!")
        starting_value += rate


    return dataframes


In [ ]:
times = 15
rate = 0.2
starting_value = 0.1

dataframes = test_loop(times, rate, starting_value)

In [ ]:
for df in dataframes:
    compute_avg_velocity_over_time(df, 100, True)
    plot_lane_occupancy(df)
    plot_average_velocity_per_lane(df)
    plot_lane_traffic_stats(df)
    